# LocEmb: Location Embedding

만든 이: 김선동 (http://seondong.github.io)

### 프로젝트 목표
* 전국 지역 단위별 embedding을 제공하고자 함
* 도로명, 상권, 사업자 등으로 확장 가능

### 사용 가능한 프로젝트
* 장소 추천/예측 모델에 pretrained된 embedding값을 활용
* COVID-19 확산/예측 SIR-based 모델 개발시 pretrained된 지역별 embedding값을 covariate으로 활용

### 사용한 알고리즘
* [NeurIPS2017](https://papers.nips.cc/paper/7213-poincare-embeddings-for-learning-hierarchical-representations)을 기반으로 다양한 feature를 추가하였음

### 제공 임베딩
* 시군구2vec: (251개 시군구 Cover, 100 dimensions): embedding+시군구명.csv
* 법정동2vec: (5005개 법정동 Cover, 100 dimensions): embedding+법정동명.csv
* 행정동2vec: (3582개 행정동 Cover, 100 dimensions): embedding+행정동명.csv
* 도로명2vec: (110722개 도로명 Cover, 100 dimensions): embedding+도로명.csv

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100

## 1. Coarse-grained Embedding: 시군구2vec

In [2]:
emb_rslt = pd.read_csv("embedding+시군구명.csv",index_col=0)

In [3]:
emb_rslt.shape

(251, 100)

In [4]:
emb_rslt.index

Index(['서울특별시 강남구', '서울특별시 금천구', '부산광역시 사상구', '서울특별시 송파구', '서울특별시 광진구', '서울특별시 강서구', '서울특별시 중구', '서울특별시 서초구', '서울특별시 성북구', '서울특별시 구로구',
       ...
       '경상북도 고령군', '전라남도 장흥군', '전라남도 영암군', '전라남도 진도군', '전라북도 부안군', '충청남도 예산군', '경상북도 영양군', '전라북도 완주군', '전라북도 무주군', '충청남도 서천군'], dtype='object', length=251)

In [5]:
emb_rslt.loc['대전광역시 유성구']

0    -0.101524
1    -0.035251
2     0.008657
3    -0.016047
4    -0.126944
        ...   
95   -0.054820
96   -0.106948
97   -0.073350
98    0.022328
99   -0.020869
Name: 대전광역시 유성구, Length: 100, dtype: float64

### 시군구 임베딩간 유사도 체크

In [6]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [7]:
print_top_n_similar('대전광역시 서구', topn=10, emb_rslt = emb_rslt)

[('대전광역시 서구', 0.0),
 ('대전광역시 중구', 0.510441405675998),
 ('충청남도 계룡시', 0.5628531213747452),
 ('대전광역시 유성구', 0.594167026715532),
 ('대전광역시 동구', 0.7465664326316799),
 ('충청남도 부여군', 0.7576599703176585),
 ('강원도 원주시', 0.7847116624779111),
 ('충청남도 금산군', 0.8112763747982337),
 ('경상북도 군위군', 0.8267369947848027),
 ('충청남도 서산시', 0.8319413861702086)]

In [8]:
print_top_n_similar('서울특별시 강남구', topn=10, emb_rslt = emb_rslt)

[('서울특별시 강남구', 0.0),
 ('서울특별시 송파구', 0.34727637437385483),
 ('서울특별시 서초구', 0.3968795838441066),
 ('경기도 성남시 수정구', 0.7272891407496148),
 ('서울특별시 용산구', 0.7415661470418881),
 ('경기도 연천군', 0.7456407562652763),
 ('울산광역시 남구', 0.7550280635048319),
 ('서울특별시 광진구', 0.7626552906740318),
 ('서울특별시 강동구', 0.7681690860050584),
 ('경기도 안성시', 0.7939815713191984)]

In [9]:
print_top_n_similar('경기도 성남시 분당구', topn=10, emb_rslt = emb_rslt)

[('경기도 성남시 분당구', 0.0),
 ('경기도 성남시 중원구', 0.3115725510763928),
 ('경기도 성남시 수정구', 0.3215208121966362),
 ('경기도 용인시 수지구', 0.35598373786239224),
 ('경기도 의왕시', 0.6206607111626793),
 ('경기도 용인시 기흥구', 0.6246114448718667),
 ('경기도 과천시', 0.6270986873771853),
 ('경기도 안양시 동안구', 0.7132537258794605),
 ('경기도 광주시', 0.7581819952552701),
 ('서울특별시 도봉구', 0.7634899771925955)]

## 2. Fine-grained Embedding: 법정동2vec

### Train된 vector 로드


In [10]:
emb_rslt = pd.read_csv("embedding+법정동명.csv",index_col=0)

In [11]:
emb_rslt.shape

(5005, 100)

In [12]:
emb_rslt.index

Index(['서울특별시 강남구 대치동', '서울특별시 강남구 역삼동', '서울특별시 금천구 독산동', '부산광역시 사상구 괘법동', '서울특별시 송파구 석촌동', '서울특별시 송파구 신천동', '서울특별시 광진구 구의동', '서울특별시 강서구 공항동', '서울특별시 중구 신당동', '서울특별시 강서구 화곡동',
       ...
       '전라북도 군산시 송창동', '전라남도 나주시 진포동', '경상북도 영천시 범어동', '경상남도 창원시 진해구 현동', '전라북도 익산시 은기동', '경상남도 창원시 진해구 대영동', '충청남도 홍성군 홍북읍', '전라북도 전주시 덕진구 중동', '경상남도 창원시 성산구 완암동', '경상북도 청송군 주왕산면'], dtype='object', length=5005)

In [13]:
emb_rslt.loc['대전광역시 유성구 도룡동']

0    -0.096376
1    -0.044024
2     0.028136
3    -0.030516
4    -0.135996
        ...   
95   -0.046185
96   -0.120554
97   -0.104367
98    0.037933
99   -0.015635
Name: 대전광역시 유성구 도룡동, Length: 100, dtype: float64

### 법정동 임베딩간 유사도 체크

In [14]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [15]:
print_top_n_similar('대전광역시 유성구 도룡동', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 도룡동', 0.0),
 ('대전광역시 유성구 덕진동', 0.0019398629391893518),
 ('대전광역시 유성구 가정동', 0.0022349192154099695),
 ('대전광역시 유성구 화암동', 0.0029246080960703136),
 ('대전광역시 유성구 장동', 0.0038995779630458527),
 ('대전광역시 유성구 자운동', 0.006284349039152204),
 ('대전광역시 유성구 추목동', 0.006316472036420606),
 ('대전광역시 유성구 신성동', 0.008692643047363502),
 ('대전광역시 유성구 신봉동', 0.009264830529820522),
 ('대전광역시 유성구 관평동', 0.010451121048860479)]

In [16]:
print_top_n_similar('서울특별시 강남구 대치동', topn=10)

[('서울특별시 강남구 대치동', 0.0),
 ('서울특별시 강남구 도곡동', 0.33942109481263416),
 ('서울특별시 강남구 일원동', 0.36272846559854965),
 ('서울특별시 강남구 개포동', 0.3776008513229916),
 ('서울특별시 강남구 수서동', 0.38990768814516796),
 ('서울특별시 강남구 삼성동', 0.43657639860766606),
 ('서울특별시 강남구 자곡동', 0.43867943895040573),
 ('서울특별시 강남구 율현동', 0.4525105455660626),
 ('서울특별시 강남구 세곡동', 0.4778294420426522),
 ('서울특별시 강남구 압구정동', 0.5725527908802119)]

In [17]:
print_top_n_similar('경기도 성남시 분당구 정자동')

[('경기도 성남시 분당구 정자동', 0.0),
 ('경기도 성남시 분당구 수내동', 0.023481452584020124),
 ('경기도 성남시 분당구 궁내동', 0.050033377236807786),
 ('경기도 성남시 분당구 금곡동', 0.05297311944970606),
 ('경기도 성남시 분당구 백현동', 0.06589377485375347),
 ('경기도 성남시 분당구 삼평동', 0.08604240826666498),
 ('경기도 성남시 분당구 율동', 0.08927583007286533),
 ('경기도 성남시 분당구 판교동', 0.08967985528334133),
 ('경기도 성남시 분당구 이매동', 0.0920953954897944),
 ('경기도 성남시 분당구 하산운동', 0.0999612154688827)]

## 3. Fine-grained Embedding: 행정동2vec

### Train된 vector 로드


In [18]:
emb_rslt = pd.read_csv("embedding+행정동명.csv",index_col=0)

In [19]:
emb_rslt.shape

(3582, 100)

In [20]:
emb_rslt.index

Index(['서울특별시 강남구 대치1동 (행정동)', '서울특별시 강남구 역삼1동 (행정동)', '서울특별시 금천구 독산3동 (행정동)', '부산광역시 사상구 괘법동 (행정동)', '서울특별시 송파구 석촌동 (행정동)', '서울특별시 송파구 잠실6동 (행정동)', '서울특별시 광진구 구의2동 (행정동)', '서울특별시 강서구 공항동 (행정동)', '서울특별시 중구 약수동 (행정동)', '서울특별시 강서구 화곡1동 (행정동)',
       ...
       '충청남도 서천군 시초면 (행정동)', '경상남도 진주시 하대동 (행정동)', '경상남도 창원시 마산회원구 석전동 (행정동)', '경상남도 진주시 상대동 (행정동)', '경상남도 창원시 마산합포구 반월중앙동 (행정동)', '충청남도 홍성군 홍북읍 (행정동)', '전라북도 전주시 완산구 효자5동 (행정동)', '전라북도 전주시 덕진구 혁신동 (행정동)', '전라남도 나주시 송월동 (행정동)', '경상북도 청송군 주왕산면 (행정동)'], dtype='object', length=3582)

In [21]:
emb_rslt.loc['대전광역시 유성구 신성동 (행정동)']

0    -0.096121
1    -0.033550
2     0.030865
3    -0.036145
4    -0.123113
        ...   
95   -0.054902
96   -0.133306
97   -0.092315
98    0.036808
99   -0.016143
Name: 대전광역시 유성구 신성동 (행정동), Length: 100, dtype: float64

### 행정동 임베딩간 유사도 체크

In [22]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [23]:
print_top_n_similar('대전광역시 유성구 신성동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 신성동 (행정동)', 0.0),
 ('대전광역시 유성구 관평동 (행정동)', 0.021876420210952152),
 ('대전광역시 유성구 전민동 (행정동)', 0.02665221205305013),
 ('대전광역시 유성구 구즉동 (행정동)', 0.03913206347886289),
 ('대전광역시 유성구 온천2동 (행정동)', 0.0505411066838366),
 ('대전광역시 서구 만년동 (행정동)', 0.06536609927745674),
 ('대전광역시 유성구 노은2동 (행정동)', 0.09147975441253464),
 ('대전광역시 유성구 노은1동 (행정동)', 0.09243195507761492),
 ('대전광역시 유성구 노은3동 (행정동)', 0.11486752536410283),
 ('대전광역시 유성구 온천1동 (행정동)', 0.18604340257623353)]

In [24]:
print_top_n_similar('경기도 성남시 수정구 신흥1동 (행정동)', topn=10)

[('경기도 성남시 수정구 신흥1동 (행정동)', 0.0),
 ('경기도 성남시 수정구 신흥3동 (행정동)', 0.0050421771603698096),
 ('경기도 성남시 수정구 신흥2동 (행정동)', 0.01880039589321747),
 ('경기도 성남시 수정구 산성동 (행정동)', 0.21619363682708947),
 ('경기도 성남시 수정구 단대동 (행정동)', 0.3215224612293076),
 ('경기도 성남시 수정구 양지동 (행정동)', 0.3431662429067742),
 ('경기도 성남시 중원구 중앙동 (행정동)', 0.3808286101928736),
 ('경기도 성남시 수정구 복정동 (행정동)', 0.4310022279868674),
 ('경기도 성남시 수정구 위례동 (행정동)', 0.43975654645503026),
 ('경기도 성남시 수정구 태평4동 (행정동)', 0.4398913106639405)]

In [25]:
print_top_n_similar('광주광역시 서구 상무1동 (행정동)', topn=10)

[('광주광역시 서구 상무1동 (행정동)', 0.0),
 ('광주광역시 서구 상무2동 (행정동)', 0.01772828709211771),
 ('광주광역시 서구 치평동 (행정동)', 0.07321158943015416),
 ('광주광역시 서구 유덕동 (행정동)', 0.08256232360853422),
 ('광주광역시 서구 화정1동 (행정동)', 0.1888402168884864),
 ('광주광역시 서구 동천동 (행정동)', 0.21308216198948593),
 ('광주광역시 서구 서창동 (행정동)', 0.23751348944146444),
 ('광주광역시 서구 금호1동 (행정동)', 0.28738024956084096),
 ('광주광역시 서구 금호2동 (행정동)', 0.3197941214617168),
 ('광주광역시 서구 광천동 (행정동)', 0.3271487901941138)]

## 4. Fine-grained Embedding: 도로명2vec

### Train된 vector 로드


In [26]:
emb_rslt = pd.read_csv("embedding+도로명.csv",index_col=0)

In [27]:
emb_rslt.shape

(110722, 100)

In [28]:
emb_rslt.index

Index(['서울특별시 강남구 남부순환로', '서울특별시 강남구 선릉로93길', '서울특별시 금천구 가산로', '부산광역시 사상구 괘감로', '서울특별시 송파구 가락로', '서울특별시 송파구 올림픽로', '서울특별시 광진구 자양로39길', '서울특별시 강서구 공항대로3길', '서울특별시 중구 동호로7길', '서울특별시 강서구 화곡로18길',
       ...
       '충청남도 서산시 학동10로', '전라북도 전주시 덕진구 가인로', '경상북도 포항시 북구 흥해읍 성곡서로18번길', '경상남도 합천군 율곡면 노양길', '경상남도 사천시 곤양면 고동포로', '전라남도 여수시 소라면 대곡해안길', '전라남도 여수시 문수북3길', '충청남도 당진시 송산면 서정6길', '충청남도 논산시 연산면 선비로646번길', '경상북도 예천군 호명면 새움1로'], dtype='object', length=110722)

In [29]:
emb_rslt.loc['대전광역시 유성구 엑스포로']

0    -0.098625
1    -0.054827
2     0.017174
3    -0.027535
4    -0.140265
        ...   
95   -0.034581
96   -0.096495
97   -0.116205
98    0.030405
99   -0.018321
Name: 대전광역시 유성구 엑스포로, Length: 100, dtype: float64

### 도로명 임베딩간 유사도 체크

In [30]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [31]:
print_top_n_similar('서울특별시 강남구 테헤란로', topn=10, emb_rslt=emb_rslt)

[('서울특별시 강남구 테헤란로', 0.0),
 ('서울특별시 강남구 테헤란로47길', 0.0275179391075574),
 ('서울특별시 강남구 선릉로91길', 0.03234538055216507),
 ('서울특별시 강남구 역삼로', 0.03452905659022287),
 ('서울특별시 강남구 선릉로99길', 0.04540327216576723),
 ('서울특별시 강남구 테헤란로55길', 0.05343564208291651),
 ('서울특별시 강남구 테헤란로51길', 0.06225098131465312),
 ('서울특별시 강남구 테헤란로57길', 0.06635382450599048),
 ('서울특별시 강남구 선릉로90길', 0.06962670096368229),
 ('서울특별시 강남구 봉은사로', 0.070340616899071)]

In [32]:
print_top_n_similar('부산광역시 사하구 낙동대로', topn=10, emb_rslt=emb_rslt)

[('부산광역시 사하구 낙동대로', 0.0),
 ('부산광역시 사하구 승학로', 0.015802638387265366),
 ('부산광역시 사하구 제석로18번길', 0.016558086339154565),
 ('부산광역시 사하구 동매로', 0.018345109001480786),
 ('부산광역시 사하구 괴정로', 0.02117502580314934),
 ('부산광역시 사하구 제석로', 0.02443476782111431),
 ('부산광역시 사하구 낙동대로436번길', 0.02894056484447638),
 ('부산광역시 사하구 낙동대로407번길', 0.032147635994370316),
 ('부산광역시 사하구 낙동대로410번길', 0.034144826206663814),
 ('부산광역시 사하구 승학로17번길', 0.038548895625728385)]

In [33]:
print_top_n_similar('대전광역시 유성구 엑스포로', topn=10)

[('대전광역시 유성구 엑스포로', 0.0),
 ('대전광역시 유성구 문지로', 0.0002718512383648175),
 ('대전광역시 유성구 전민로', 0.0003844291641329489),
 ('대전광역시 유성구 엑스포로540번길', 0.0007245344781847685),
 ('대전광역시 유성구 엑스포로466번길', 0.000832637142804793),
 ('대전광역시 유성구 엑스포로538번길', 0.0008356207778184022),
 ('대전광역시 유성구 유성대로1689번길', 0.000885434275087249),
 ('대전광역시 유성구 엑스포로446번길', 0.0008893497645806603),
 ('대전광역시 유성구 엑스포로295번길', 0.0009518560485280991),
 ('대전광역시 유성구 유성대로1628번길', 0.0014761716780355982)]